<a href="https://colab.research.google.com/github/EmmaHsueh/PL_project/blob/main/Simple_Expense_Tracker(%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Expense Tracker
目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。
AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。
Sheet 欄位：date, category, item, amount, payer

https://docs.google.com/spreadsheets/d/1IbbAn-J65AR1CDjqmtNR4IvOmmhwyJoNPg5pAgrrFXI/edit?gid=0#gid=0


In [7]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [8]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1IbbAn-J65AR1CDjqmtNR4IvOmmhwyJoNPg5pAgrrFXI/edit?gid=0#gid=0')

## **記錄支出**

In [9]:
import datetime
import pandas as pd # Import pandas

# 打開你的試算表
sheet_name = '消費紀錄' # 請換成你的試算表名稱
# sheet = gc.open(sheet_name).sheet1 # 移除這行，因為我們已經用 URL 開啟試算表
sheet = gsheets.sheet1 # 使用 gsheets 變數來取得第一個工作表 (通常是 sheet1)

# Initialize df_expenses as an empty DataFrame with the correct columns
df_expenses = pd.DataFrame(columns=['日期', '類別', '品項', '金額', '付款方式'])

def record_expense():
    """
    記錄一筆日常支出並添加到 DataFrame
    """
    global df_expenses # Declare df_expenses as global to modify it

    date_str = input("請輸入日期 (YYYY-MM-DD)：")
    item = input("請輸入品項：")
    amount = float(input("請輸入金額："))
    category = input("請輸入類別：")
    payer = input("請輸入付款方式：")

    # 建立新的資料列 (作為 DataFrame 的一行)
    new_expense = pd.DataFrame([[date_str, category, item, amount, payer]],
                               columns=['日期', '類別', '品項', '金額', '付款方式']) # Ensure column names match

    # 將新的資料列添加到現有的 DataFrame
    df_expenses = pd.concat([df_expenses, new_expense], ignore_index=True)

    print(f"成功記錄並添加到 DataFrame：日期={date_str}, 類別={category}, 品項={item}, 金額={amount}, 付款方式={payer}")
    # We will write the DataFrame to the sheet in a separate step

# 執行記錄功能 (現在只會添加到 DataFrame)
# record_expense() # 先註解掉這行，避免重複執行

data = gsheets.sheet1.get_all_values()
df_expenses = pd.DataFrame(data[1:], columns=data[0])
display(df_expenses)

,日期,時間,品項,金額,
0,2025-09-11,daily,coffee,45,cash
1,2025-09-11,daily,lunch,120,cash
2,2025-09-11,daily,dinner,120,cash
3,2025-09-11,daily,dinner,130,cash
4,2025-9-25,none,apple,10,line pay
5,2025-09-25,daily,coffee,45,cash
6,2025-09-25,daily,coffee,45,cash


## **計算支出總額與分攤人數**

In [10]:
def calculate_share():
    """
    讀取Google Sheets數據並計算分攤
    """
    # 讀取所有數據
    data = sheet.get_all_records()

    if not data:
        print("沒有任何支出記錄。")
        return

    # 計算總金額
    total_amount = sum(row['金額'] for row in data)

    # 詢問是否需要計算分攤金額
    calculate_split = input("需要計算分攤金額嗎？(yes/no): ").lower()

    if calculate_split != 'yes':
        print(f"總支出金額：{total_amount:.2f} 元")
        print("不計算分攤金額。")
        return

    # 讓使用者輸入分攤人數
    while True:
        try:
            num_people_str = input("請輸入分攤人數：")
            num_people = int(num_people_str)
            if num_people <= 0:
                print("分攤人數必須大於零。")
            else:
                break
        except ValueError:
            print("請輸入有效的數字。")

    if num_people > 0:
        share_per_person = total_amount / num_people
        print("\n--- 支出分攤速算 ---")
        print(f"總支出金額：{total_amount:.2f} 元")
        print(f"每人應分攤金額（假設 {num_people} 人）：{share_per_person:.2f} 元")
    else:
        print("無法計算分攤金額，因為分攤人數為零。")

# 執行分攤計算
calculate_share() # 先註解掉這行，避免重複執行

需要計算分攤金額嗎？(yes/no): no
總支出金額：515.00 元
不計算分攤金額。


In [11]:
# Clear the existing data in the sheet (optional, but ensures the sheet matches the DataFrame)
gsheets.sheet1.clear()

# Write the DataFrame to the sheet, including headers
gsheets.sheet1.update([df_expenses.columns.values.tolist()] + df_expenses.values.tolist())

print("DataFrame 已成功寫回 Google Sheet。")

DataFrame 已成功寫回 Google Sheet。


## **總程式**

In [6]:
def main():
    while True:
        print("\n--- 支出記錄與分攤程式 ---")
        print("1. 記錄一筆支出")
        print("2. 計算支出總額與分攤")
        print("3. 退出")

        choice = input("請選擇功能（1/2/3）：")

        if choice == '1':
            record_expense()

            data = gsheets.sheet1.get_all_values()
            df_expenses = pd.DataFrame(data[1:], columns=data[0])
            display(df_expenses)
        elif choice == '2':
            calculate_share()
        elif choice == '3':
            print("程式已退出。")
            break
        else:
            print("無效的選擇，請重新輸入。")

# 運行主程式
main()


--- 支出記錄與分攤程式 ---
1. 記錄一筆支出
2. 計算支出總額與分攤
3. 退出
請選擇功能（1/2/3）：1
請輸入日期 (YYYY-MM-DD)：2025-10-4
請輸入品項：coffee
請輸入金額：45
請輸入類別：daily
請輸入付款方式：cash
成功記錄並添加到 DataFrame：日期=2025-10-4, 類別=daily, 品項=coffee, 金額=45.0, 付款方式=cash


,日期,時間,品項,金額,
0,2025-09-11,daily,coffee,45,cash
1,2025-09-11,daily,lunch,120,cash
2,2025-09-11,daily,dinner,120,cash
3,2025-09-11,daily,dinner,130,cash
4,2025-9-25,none,apple,10,line pay
5,2025-09-25,daily,coffee,45,cash
6,2025-09-25,daily,coffee,45,cash



--- 支出記錄與分攤程式 ---
1. 記錄一筆支出
2. 計算支出總額與分攤
3. 退出
請選擇功能（1/2/3）：3
程式已退出。
